In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
# 將訓練資料讀取到資料框架
train = pd.read_csv("../input/house-prices-advanced-regression-techniques/train.csv")
# 將測試資料讀取到資料框架
test = pd.read_csv("../input/house-prices-advanced-regression-techniques/test.csv")


In [ ]:
import numpy as np
from scipy.stats import skew

# 將SalePrice做對數轉換
prices = pd.DataFrame({'price':train['SalePrice']})
train["SalePrice"] = np.log1p(train["SalePrice"])

# 合併訓練資料與測試資料中僅的MSSubClass～SaleCondition欄位
all_data = pd.concat((train.loc[:,'MSSubClass':'SaleCondition'],
                      test.loc[:,'MSSubClass':'SaleCondition']))

# 取得非object類型欄位的index
numeric_feats = all_data.dtypes[all_data.dtypes != "object"].index

# 填補非object類型欄位的缺失值，並且計算偏度
skewed_feats = train[numeric_feats].apply(lambda x: skew(x.dropna()))

# 留下偏度大於0.75的欄位
skewed_feats = skewed_feats[skewed_feats > 0.75]

# 取出留下欄位的index
skewed_feats = skewed_feats.index

# 將偏度大於0.75的欄位做對數變換
all_data[skewed_feats] = np.log1p(all_data[skewed_feats])

# 將LotShape（土地形狀）進行One-hot encoding
cc_data = pd.get_dummies(train['LotShape'])
# 與原本LotShape資料合併
cc_data['LotShape'] = train['LotShape']

# 將類別變數進行One-hot encoding
all_data = pd.get_dummies(all_data)

# 將缺失值換成為該行的平均值（訓練資料當中的平均值）
all_data = all_data.fillna(all_data[:train.shape[0]].mean())

# 分割出訓練資料與測試資料
X_train = all_data[:train.shape[0]]
X_test = all_data[train.shape[0]:]
y = train.SalePrice



In [ ]:
from sklearn.model_selection import cross_val_score
import xgboost as xgb

def rmse_cv(model):
    """
    均方根誤差
    Parameters:
    model(obj): Model object
    Returns:(float)訓練資料的預測值與正確值的均方誤差(RMSE)
    """
    #  透過交叉驗證取得均方根誤差
    rmse = np.sqrt(-cross_val_score(model, 
                                    X_train, 
                                    y,
                                    scoring="neg_mean_squared_error",
                                    cv = 5))
    return(rmse)

# 使用xgbboost進行學習
model_xgb = xgb.XGBRegressor(n_estimators=410,  #決策樹的數量
                             max_depth=3,       #決策樹的深度
                             learning_rate=0.1) #學習率0.1

model_xgb.fit(X_train, y)

print ('xgboost RMSE loss: ')
print (rmse_cv(model_xgb) .mean())


In [ ]:
preds = np.expm1(model_xgb.predict(X_test))
solution = pd.DataFrame({"id":test.Id, "SalePrice":preds})
solution.to_csv("ridge_sol.csv", index = False)

In [ ]:
print(preds)